In [38]:
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score
from transformers import DistilBertModel, DistilBertTokenizerFast, get_linear_schedule_with_warmup
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import ast

In [39]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device

device(type='mps')

In [40]:
data_file = "../movie_dataset_prepared.csv"
df = pd.read_csv(data_file)

In [41]:
df['genres_list'] = df['genres_list'].apply(ast.literal_eval)
df['keywords_list'] = df['keywords_list'].apply(ast.literal_eval)

print(type(df['genres_list'][0]))
df.head()


<class 'list'>


,genres_list,keywords_list,overview
0,"[28, 12, 14, 878]","[9951, 209714]",in the 22nd century a paraplegic marine is dis...
1,"[12, 14, 28]",[179430],captain barbossa long believed to be dead has ...
2,"[28, 12, 80]","[818, 9663]",a cryptic message from bonds past sends him on...
3,"[28, 80, 18, 53]",[9715],following the death of district attorney harve...
4,"[28, 12, 878]","[818, 9951, 10685, 209714]",john carter is a warweary former military capt...


In [42]:
def clean_overview(x):
    if isinstance(x, str):
        return x
    if pd.isna(x):
        return ""
    return str(x)

df['overview'] = df['overview'].apply(clean_overview)

In [43]:
print(f"Przed usunieciem pustych keywords: {len(df)}")
df = df[df['keywords_list'].apply(lambda x: len(x) > 0)]
print(f"Po usunieciu pustych keywords: {len(df)}")

Przed usunieciem pustych keywords: 4683
Po usunieciu pustych keywords: 2720


In [44]:
texts = df['overview'].tolist() 
genre_labels = df['genres_list'].tolist()
keywords_labels = df['keywords_list'].tolist()

genre_labels, keywords_labels

([[28, 12, 14, 878],
  [12, 14, 28],
  [28, 12, 80],
  [28, 80, 18, 53],
  [28, 12, 878],
  [14, 28, 12],
  [16, 10751],
  [28, 12, 878],
  [12, 14, 10751],
  [28, 12, 14],
  [12, 14, 28, 878],
  [12, 14, 28],
  [28, 12],
  [28, 12, 14, 878],
  [12, 10751, 14],
  [878, 28, 12],
  [12, 28, 14],
  [28, 35, 878],
  [28, 12, 14],
  [28, 12, 14],
  [18, 10749, 53],
  [12, 28, 878],
  [53, 28, 12, 878],
  [28, 12, 878, 53],
  [28, 12, 14],
  [28, 12, 878],
  [10751, 14, 12],
  [12, 28, 878, 53],
  [16, 10751],
  [878, 28, 12],
  [878, 28, 12],
  [14, 12, 10751],
  [28, 12, 14],
  [12, 28, 878],
  [16, 10751, 12, 35],
  [12, 28, 53, 878],
  [16, 10751, 35],
  [28, 878, 53],
  [28],
  [28, 18, 27, 878, 53],
  [28, 12, 14, 878],
  [28, 12, 878],
  [18, 10749],
  [28, 878, 12],
  [28, 878, 12],
  [12, 16, 10751],
  [16, 12, 35, 10751, 28, 14],
  [28, 12, 878],
  [16, 18],
  [878, 14, 28, 12],
  [12, 10751, 14],
  [878],
  [18, 28, 80, 53],
  [16, 35, 10751, 12],
  [16, 10751, 12, 878],
  [28, 87

In [123]:
mlb = MultiLabelBinarizer()
keywords_labels_encoded = mlb.fit_transform(keywords_labels)
keywords_n_classes = len(mlb.classes_)

genres_labels_encoded = mlb.fit_transform(genre_labels)
genres_n_classes = len(mlb.classes_)

In [124]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

In [125]:
class MovieMultiTaskDataset(Dataset):
    def __init__(self, texts, genre_labels, keyword_labels, tokenizer, max_length=256):
        self.texts = texts
        self.genre_labels = genre_labels
        self.keyword_labels = keyword_labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        genre_label = self.genre_labels[idx]
        keyword_label = self.keyword_labels[idx]

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'genre_labels': torch.FloatTensor(genre_label),
            'keyword_labels': torch.FloatTensor(keyword_label)
        }

In [126]:
class BERTMultiTaskClassifier(nn.Module):
    def __init__(self, n_genres, n_keywords, dropout=0.3):
        super().__init__()
        self.bert = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.dropout = nn.Dropout(dropout)
        self.genre_classifier = nn.Linear(self.bert.config.hidden_size, n_genres)
        self.keyword_classifier = nn.Linear(self.bert.config.hidden_size, n_keywords)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # CLS token
        pooled_output = self.dropout(pooled_output)
        genre_logits = self.genre_classifier(pooled_output)
        keyword_logits = self.keyword_classifier(pooled_output)
        return genre_logits, keyword_logits

In [127]:
def train_epoch(model, data_loader, genre_criterion, keyword_criterion, optimizer, scheduler, device, genre_weight=1.0, keyword_weight=1.0):
    model.train()
    total_loss = 0

    for batch in tqdm(data_loader, desc='Training'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        genre_labels = batch['genre_labels'].to(device)
        keyword_labels = batch['keyword_labels'].to(device)

        optimizer.zero_grad()
        genre_logits, keyword_logits = model(input_ids, attention_mask)

        loss = genre_weight * genre_criterion(genre_logits, genre_labels) + \
               keyword_weight * keyword_criterion(keyword_logits, keyword_labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        total_loss += loss.item()
    return total_loss / len(data_loader)

In [150]:
from sklearn.metrics import f1_score, precision_score, recall_score, hamming_loss, jaccard_score

@torch.no_grad()
def evaluate(model, data_loader, genre_criterion, keyword_criterion, device,
                       genre_thresh=0.3, keyword_thresh=0.5):
    model.eval()
    all_genre_preds, all_genre_labels = [], []
    all_keyword_preds, all_keyword_labels = [], []
    total_loss = 0

    for batch in tqdm(data_loader, desc='Evaluating'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        genre_labels = batch['genre_labels'].to(device)
        keyword_labels = batch['keyword_labels'].to(device)

        genre_logits, keyword_logits = model(input_ids, attention_mask)
        loss = genre_criterion(genre_logits, genre_labels) + keyword_criterion(keyword_logits, keyword_labels)
        total_loss += loss.item()

        genre_preds = (torch.sigmoid(genre_logits) > genre_thresh).cpu().numpy()
        keyword_preds = (torch.sigmoid(keyword_logits) > keyword_thresh).cpu().numpy()

        all_genre_preds.extend(genre_preds)
        all_genre_labels.extend(genre_labels.cpu().numpy())
        all_keyword_preds.extend(keyword_preds)
        all_keyword_labels.extend(keyword_labels.cpu().numpy())

    all_genre_preds = np.array(all_genre_preds)
    all_genre_labels = np.array(all_genre_labels)
    all_keyword_preds = np.array(all_keyword_preds)
    all_keyword_labels = np.array(all_keyword_labels)

    metrics = {
        'loss': total_loss / len(data_loader),

        # Genre metrics
        'genre_f1_score': f1_score(all_genre_labels, all_genre_preds, average='samples', zero_division=0),
        'genre_f1_micro': f1_score(all_genre_labels, all_genre_preds, average='micro', zero_division=0),
        'genre_f1_macro': f1_score(all_genre_labels, all_genre_preds, average='macro', zero_division=0),
        'genre_precision_micro': precision_score(all_genre_labels, all_genre_preds, average='micro', zero_division=0),
        'genre_recall_micro': recall_score(all_genre_labels, all_genre_preds, average='micro', zero_division=0),
        'genre_hamming_loss': hamming_loss(all_genre_labels, all_genre_preds),
        'genre_hamming_accuracy': 1 -hamming_loss(all_genre_labels, all_genre_preds),
        'genre_jaccard_samples': jaccard_score(all_genre_labels, all_genre_preds, average='samples'),

        # Keyword metrics
        'keyword_f1_score': f1_score(all_keyword_labels, all_keyword_preds, average='samples', zero_division=0),
        'keyword_f1_micro': f1_score(all_keyword_labels, all_keyword_preds, average='micro', zero_division=0),
        'keyword_f1_macro': f1_score(all_keyword_labels, all_keyword_preds, average='macro', zero_division=0),
        'keyword_precision_micro': precision_score(all_keyword_labels, all_keyword_preds, average='micro', zero_division=0),
        'keyword_recall_micro': recall_score(all_keyword_labels, all_keyword_preds, average='micro', zero_division=0),
        'keyword_hamming_loss': hamming_loss(all_keyword_labels, all_keyword_preds),
        'keyword_hamming_accuracy': 1 - hamming_loss(all_keyword_labels, all_keyword_preds),
        'keyword_jaccard_samples': jaccard_score(all_keyword_labels, all_keyword_preds, average='samples')
    }

    return metrics


In [129]:
def train_model_multitask(
    texts,
    genre_labels_encoded,
    keyword_labels_encoded,
    genres_n_classes,
    keywords_n_classes,
    epochs=15,
    batch_size=16,
    learning_rate=3e-5,
    max_length=256,
    val_size=0.2,
    genre_threshold=0.3,
    keyword_threshold=0.5,
    model_name="multitask",
    patience=3
):
    # ===== CLASS WEIGHTS =====
    genre_pos_weight = torch.tensor((len(genre_labels_encoded) - genre_labels_encoded.sum(axis=0)) / np.clip(genre_labels_encoded.sum(axis=0), 1, None), dtype=torch.float).to(device)
    keyword_pos_weight = torch.tensor((len(keyword_labels_encoded) - keyword_labels_encoded.sum(axis=0)) / np.clip(keyword_labels_encoded.sum(axis=0), 1, None), dtype=torch.float).to(device)

    # ===== SPLIT =====
    X_train, X_val, y_genre_train, y_genre_val, y_keyword_train, y_keyword_val = train_test_split(
        texts, genre_labels_encoded, keyword_labels_encoded, test_size=val_size, random_state=42, shuffle=True
    )

    # ===== DATASETS =====
    train_loader = DataLoader(MovieMultiTaskDataset(X_train, y_genre_train, y_keyword_train, tokenizer, max_length), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(MovieMultiTaskDataset(X_val, y_genre_val, y_keyword_val, tokenizer, max_length), batch_size=batch_size)

    # ===== MODEL =====
    model = BERTMultiTaskClassifier(n_genres=genres_n_classes, n_keywords=keywords_n_classes).to(device)
    genre_criterion = nn.BCEWithLogitsLoss(pos_weight=genre_pos_weight)
    keyword_criterion = nn.BCEWithLogitsLoss(pos_weight=keyword_pos_weight)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader)*epochs)

    history = {"train_loss": []}

    best_score = 0.0
    wait = 0

    for epoch in range(epochs):
        # ---- TRAIN ----
        train_loss = train_epoch(model, train_loader, genre_criterion, keyword_criterion, optimizer, scheduler, device)
        val_metrics = evaluate(model, val_loader, genre_criterion, keyword_criterion, device, genre_thresh=genre_threshold, keyword_thresh=keyword_threshold)

        # ---- AUTOMATYCZNE DODANIE WSZYSTKICH METRYK ----
        history["train_loss"].append(train_loss)
        for k, v in val_metrics.items():
            if k not in history:
                history[k] = []
            history[k].append(v)

        # ---- EARLY STOPPING ----
        score = 0.5 * (val_metrics['genre_f1_micro'] + val_metrics['keyword_f1_micro'])
        if score > best_score:
            best_score = score
            wait = 0
            torch.save(model.state_dict(), f"{model_name}.pth")
        else:
            wait += 1
            if wait >= patience:
                print(f"\nEarly stopping po {epoch+1} epokach (najlepszy wynik: {best_score:.4f})")
                break

        print(f"Epoch {epoch+1}: Loss={train_loss:.4f}, Genre F1={val_metrics['genre_f1_micro']:.4f}, Keyword F1={val_metrics['keyword_f1_micro']:.4f}")

    # ---- LOAD BEST MODEL ----
    model.load_state_dict(torch.load(f"{model_name}.pth"))

    return {
        "model": model,
        "history": history
    }


In [131]:
result = train_model_multitask(
    texts,
    genres_labels_encoded,
    keywords_labels_encoded,
    genres_n_classes,
    keywords_n_classes,
    epochs=1,
    batch_size=16,
    learning_rate=3e-5
)

model = result["model"]


Evaluating: 100%|██████████| 34/34 [00:06<00:00,  5.33it/s]


Epoch 1: Loss=2.3850, Genre F1=0.3103, Keyword F1=0.0917


## PARAM GRID

In [69]:
param_grid = {
    "batch_size": [8, 16],
    "learning_rate": [2e-5, 3e-5],
    "max_length": [128, 256],
    "epochs": [3],
    "genre_threshold": [0.3, 0.35],
    "keyword_threshold": [0.4, 0.5]
}

In [81]:
from itertools import product
import pandas as pd
import os

def param_grid_search_multitask(
    texts,
    genre_labels,
    keyword_labels,
    param_grid,
    subset_frac=0.2,  # <- nowy parametr, 1.0 = pełne dane
    save_csv_path="multitask_param_grid_results.csv",
    model_prefix="param_grid/multitask",
):
    keys = list(param_grid.keys())
    combinations = list(product(*param_grid.values()))

    results = []
    
    if subset_frac < 1.0:
        texts, texts_unused, genre_labels, genre_labels_unused, keyword_labels, keyword_labels_unused = train_test_split(
            texts, genre_labels, keyword_labels,
            test_size=1-subset_frac,
            random_state=42,
            shuffle=True
        )


    print(f"Running {len(combinations)} multitask configurations...\n")

    for i, values in enumerate(combinations, 1):
        config = dict(zip(keys, values))

        print("=" * 70)
        print(f"[{i}/{len(combinations)}] CONFIG: {config}")
        print("=" * 70)

        run = train_model_multitask(
            texts=texts,
            genre_labels=genre_labels,
            keyword_labels=keyword_labels,
            epochs=config["epochs"],
            batch_size=config["batch_size"],
            learning_rate=config["learning_rate"],
            max_length=config["max_length"],
            genre_threshold=config["genre_threshold"],
            keyword_threshold=config["keyword_threshold"],
            model_name=f"{model_prefix}_{i}"
        )

        history = run["history"]

        # Ostatnie metryki walidacyjne
        row = {
            **config,
            "genre_f1_micro": history["genre_f1_micro"][-1],
            "genre_f1_macro": history["genre_f1_macro"][-1],
            "keyword_f1_micro": history["keyword_f1_micro"][-1],
            "keyword_f1_macro": history["keyword_f1_macro"][-1],
        }

        row["mean_f1_micro"] = (
            row["genre_f1_micro"] + row["keyword_f1_micro"]
        ) / 2

        results.append(row)

        # zapis po KAŻDEJ konfiguracji (bezpieczne)
        pd.DataFrame(results).to_csv(save_csv_path, index=False)

        print(
            f"Genre F1: {row['genre_f1_micro']:.4f} | "
            f"Keyword F1: {row['keyword_f1_micro']:.4f} | "
            f"Mean F1: {row['mean_f1_micro']:.4f}\n"
        )

    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values("mean_f1_micro", ascending=False)

    print("\n===== TOP 5 CONFIGURATIONS =====")
    print(results_df.head())

    return results_df


In [72]:
results_df = param_grid_search_multitask(
    texts=texts,
    genre_labels=genre_labels,
    keyword_labels=keywords_labels,
    param_grid=param_grid,
    save_csv_path="param_grid/multitask_grid_results.csv"
)


Running 32 multitask configurations...

[1/32] CONFIG: {'batch_size': 8, 'learning_rate': 2e-05, 'max_length': 128, 'epochs': 3, 'genre_threshold': 0.3, 'keyword_threshold': 0.4}


Evaluating: 100%|██████████| 14/14 [00:00<00:00, 22.90it/s]


Epoch 1: Loss 2.5080, Genre F1 0.2942, Keyword F1 0.0650


Evaluating: 100%|██████████| 14/14 [00:00<00:00, 22.74it/s]


Epoch 2: Loss 2.3999, Genre F1 0.2947, Keyword F1 0.0670


...

Epoch 1: Loss 2.4939, Genre F1 0.2942, Keyword F1 0.0734


Evaluating: 100%|██████████| 7/7 [00:01<00:00,  6.16it/s]


Epoch 2: Loss 2.4330, Genre F1 0.2964, Keyword F1 0.0732


Evaluating: 100%|██████████| 7/7 [00:01<00:00,  5.46it/s]


Epoch 3: Loss 2.2992, Genre F1 0.3035, Keyword F1 0.0739
Genre F1: 0.3035 | Keyword F1: 0.0739 | Mean F1: 0.1887


===== TOP 5 CONFIGURATIONS =====
    batch_size  learning_rate  max_length  epochs  genre_threshold  \
11           8        0.00003         128       3             0.35   
13           8        0.00003         256       3             0.30   
14           8        0.00003         256       3             0.35   
15           8        0.00003         256       3             0.35   
10           8        0.00003         128       3             0.35   

    keyword_threshold  genre_f1_micro  genre_f1_macro  keyword_f1_micro  \
11                0.5        0.405993        0.382104          0.107452   
13                0.5        0.347283        0.329544          0.094417   
14                0.4        0.364829        0.346388          0.074578   
15                0.5        0.344313        0.320179          0.092359   
10                0.4        0.361057        0.339436   

In [ ]:
def select_optimal_epochs_multitask(
    texts,
    genre_labels_encoded,
    keyword_labels_encoded,
    tokenizer,
    device,
    max_length=256,
    batch_size=16,
    learning_rate=3e-5,
    max_epochs=20,
    patience=3,
    genre_loss_weight=1.0,
    keyword_loss_weight=1.0,
    genre_thresh=0.35,
    keyword_thresh=0.5,
    val_size=0.2
):
    """
    Dobór liczby epok z early stopping dla modelu multi-task (genres + keywords)
    """

    n_genres = genre_labels_encoded.shape[1]   # liczba gatunków

    n_keywords = keyword_labels_encoded.shape[1]  # liczba słów kluczowych
    # Podział na train/val
    X_train, X_val, y_genre_train, y_genre_val, y_keyword_train, y_keyword_val = train_test_split(
        texts, genre_labels_encoded, keyword_labels_encoded,
        test_size=val_size,
        random_state=42,
        shuffle=True
    )
    
    # Dataset i DataLoader
    train_dataset = MovieMultiTaskDataset(
        texts=X_train,
        genre_labels=y_genre_train,
        keyword_labels=y_keyword_train,
        tokenizer=tokenizer,
        max_length=max_length
    )
    val_dataset = MovieMultiTaskDataset(
        texts=X_val,
        genre_labels=y_genre_val,
        keyword_labels=y_keyword_val,
        tokenizer=tokenizer,
        max_length=max_length
    )

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Model
    model = BERTMultiTaskClassifier(n_genres=n_genres, n_keywords=n_keywords).to(device)

    # Loss i optymalizator
    genre_criterion = torch.nn.BCEWithLogitsLoss()
    keyword_criterion = torch.nn.BCEWithLogitsLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

    best_val_f1 = 0
    best_epoch = 0
    wait = 0

    print("=== Dobór liczby epok z early stopping ===\n")
    
    for epoch in range(1, max_epochs + 1):
        # ---- trening ----
        model.train()
        train_loss = 0
        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch}"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            genre_labels_batch = batch['genre_labels'].to(device)
            keyword_labels_batch = batch['keyword_labels'].to(device)

            optimizer.zero_grad()
            genre_logits, keyword_logits = model(input_ids, attention_mask)

            loss = genre_loss_weight * genre_criterion(genre_logits, genre_labels_batch) + \
                   keyword_loss_weight * keyword_criterion(keyword_logits, keyword_labels_batch)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)

        # ---- walidacja ----
        val_metrics = evaluate(
            model, val_loader, genre_criterion, keyword_criterion,
            device=device, genre_thresh=genre_thresh, keyword_thresh=keyword_thresh
        )

        val_f1_micro = 0.5 * (val_metrics['genre_f1_micro'] + val_metrics['keyword_f1_micro'])

        print(f"Epoch {epoch}: Train Loss={train_loss:.4f}, Val F1 Micro={val_f1_micro:.4f}")

        # ---- early stopping ----
        if val_f1_micro > best_val_f1:
            best_val_f1 = val_f1_micro
            best_epoch = epoch
            wait = 0
            torch.save(model.state_dict(), "best_multitask_epoch.pth")
        else:
            wait += 1
            if wait >= patience:
                print(f"\nEarly stopping po {epoch} epokach (najlepsza epoka {best_epoch})")
                break

    print(f"\nOptymalna liczba epok: {best_epoch}, najlepsze F1 Micro: {best_val_f1:.4f}")
    return best_epoch, best_val_f1


In [ ]:
best_epoch, best_f1= select_optimal_epochs_multitask(
    texts=texts,
    genres_labels_encoded=genres_labels_encoded,
    keywords_labels_encoded=keywords_labels_encoded,
    tokenizer=tokenizer, 
    device=device,
    max_length=128,
    batch_size=8,
    learning_rate=3e-5,
    max_epochs=20,          # maksymalna liczba epok do sprawdzenia
    patience=3,             # early stopping po 3 epokach bez poprawy
    genre_loss_weight=1.0,
    keyword_loss_weight=1.0,
    genre_thresh=0.35,
    keyword_thresh=0.5,
    val_size=0.2
)


=== Dobór liczby epok z early stopping ===



Evaluating: 100%|██████████| 68/68 [00:03<00:00, 22.18it/s]


Epoch 1: Train Loss=0.6215, Val F1 Micro=0.2984


Evaluating: 100%|██████████| 68/68 [00:03<00:00, 22.22it/s]


Epoch 2: Train Loss=0.4379, Val F1 Micro=0.3217


...

Epoch 18: Train Loss=0.0925, Val F1 Micro=0.4087


Evaluating: 100%|██████████| 68/68 [00:02<00:00, 22.91it/s]


Epoch 19: Train Loss=0.0858, Val F1 Micro=0.4049


Evaluating: 100%|██████████| 68/68 [00:02<00:00, 23.26it/s]

Epoch 20: Train Loss=0.0787, Val F1 Micro=0.4083

Optymalna liczba epok: 18, najlepsze F1 Micro: 0.4087


In [145]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from sklearn.model_selection import KFold

def five_fold_cv_multitask_save_config(
    texts,
    genres_labels_encoded,
    keywords_labels_encoded,
    genres_n_classes,
    keywords_n_classes,
    tokenizer,
    device,
    config,
    n_splits=5,
    metrics_csv_path="5fold_multitask_metrics.csv",
    model_save_prefix="multitask_fold"
):
    """
    5-fold CV dla multitask BERT z wywołaniem funkcji train_model_multitask,
    early stopping, zapis modeli i metryk.
    
    config - słownik z parametrami treningu, np.:
        {
            "epochs": 5,
            "batch_size": 8,
            "learning_rate": 3e-5,
            "max_length": 256,
            "genre_loss_weight": 1.0,
            "keyword_loss_weight": 1.0,
            "genre_threshold": 0.35,
            "keyword_threshold": 0.5,
            "patience": 2
        }
    """
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    fold_results = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(texts)):
        print(f"\n=== Fold {fold+1}/{n_splits} ===")

        # ===== Podział danych =====
        X_train = [texts[i] for i in train_idx]
        X_val   = [texts[i] for i in val_idx]
        y_genre_train = genres_labels_encoded[train_idx]
        y_genre_val   = genres_labels_encoded[val_idx]
        y_keyword_train = keywords_labels_encoded[train_idx]
        y_keyword_val   = keywords_labels_encoded[val_idx]

        # ===== Trening foldu przez train_model_multitask =====
        run = train_model_multitask(
            texts=X_train,
            genre_labels_encoded=y_genre_train,
            keyword_labels_encoded=y_keyword_train,
            genres_n_classes=genres_n_classes,
            keywords_n_classes=keywords_n_classes,
            epochs=config.get("epochs", 5),
            batch_size=config.get("batch_size", 8),
            learning_rate=config.get("learning_rate", 3e-5),
            max_length=config.get("max_length", 256),
            val_size=None,  # walidacja fold-specific
            genre_threshold=config.get("genre_threshold", 0.35),
            keyword_threshold=config.get("keyword_threshold", 0.5),
            model_name=f"{model_save_prefix}_fold{fold+1}",
            patience=config.get("patience", 3)
        )

        model = run["model"]
        history = run["history"]

        # ===== Walidacja foldu =====
        val_ds = MovieMultiTaskDataset(
            texts=X_val,
            genre_labels=y_genre_val,
            keyword_labels=y_keyword_val,
            tokenizer=tokenizer,
            max_length=config.get("max_length", 256)
        )
        val_loader = DataLoader(val_ds, batch_size=config.get("batch_size", 8))

        # Loss z pos_weight (dla metryk)
        genre_pos_weight = torch.tensor(
            (len(y_genre_train) - y_genre_train.sum(axis=0)) / np.clip(y_genre_train.sum(axis=0), 1, None),
            dtype=torch.float
        ).to(device)
        keyword_pos_weight = torch.tensor(
            (len(y_keyword_train) - y_keyword_train.sum(axis=0)) / np.clip(y_keyword_train.sum(axis=0), 1, None),
            dtype=torch.float
        ).to(device)

        genre_criterion = torch.nn.BCEWithLogitsLoss(pos_weight=genre_pos_weight)
        keyword_criterion = torch.nn.BCEWithLogitsLoss(pos_weight=keyword_pos_weight)

        val_metrics = evaluate(
            model,
            val_loader,
            genre_criterion,
            keyword_criterion,
            device=device,
            genre_thresh=config.get("genre_threshold", 0.35),
            keyword_thresh=config.get("keyword_threshold", 0.5)
        )

        # Dodaj informacje o foldzie i najlepszej epoce
        val_metrics["fold"] = fold + 1
        val_metrics["best_epoch"] = len(history["train_loss"])
        fold_results.append(val_metrics)

        print(f"Fold {fold+1} - Genre F1 Micro: {val_metrics['genre_f1_micro']:.4f}, "
              f"Keyword F1 Micro: {val_metrics['keyword_f1_micro']:.4f}")

    # ===== Zapis CSV =====
    df_results = pd.DataFrame(fold_results)
    df_results.to_csv(metrics_csv_path, index=False)
    print(f"\nZapisano metryki wszystkich foldów do: {metrics_csv_path}")

    # ===== Średnie po 5 foldach =====
    avg_metrics = {}
    metric_keys = [k for k in fold_results[0].keys() if k not in ["fold", "best_epoch"]]
    for key in metric_keys:
        avg_metrics[key] = np.mean([fold[key] for fold in fold_results])

    print("\n=== Średnie wyniki 5-fold CV ===")
    for key, value in avg_metrics.items():
        print(f"{key}: {value:.4f}")

    return df_results, avg_metrics


In [ ]:
best_config = {
    "batch_size": 8,
    "learning_rate": 3e-5,
    "max_length": 256,
    "epochs": 10,
    "patience": 3,
    "genre_loss_weight": 1.0,
    "keyword_loss_weight": 1.0,
    "genre_thresh": 0.35,
    "keyword_thresh": 0.5
}

In [152]:
df_results, avg_metrics = five_fold_cv_multitask_save_config(
    texts=texts,
    genres_labels_encoded=genres_labels_encoded,
    keywords_labels_encoded=keywords_labels_encoded,
    genres_n_classes=genres_n_classes,
    keywords_n_classes=keywords_n_classes,
    tokenizer=tokenizer,
    device=device,
    config=best_config,
    n_splits=5,
    metrics_csv_path="results/cross_val_multitask_metrics.csv",
    model_save_prefix="models/multitask"
)

print("\nŚrednie wyniki po 5-fold CV:")
for key, value in avg_metrics.items():
    print(f"{key}: {value:.4f}")



=== Fold 1/5 ===


Evaluating: 100%|██████████| 68/68 [00:06<00:00, 10.49it/s]


Epoch 1: Loss=2.3023, Genre F1=0.4382, Keyword F1=0.1236


Evaluating: 100%|██████████| 68/68 [00:06<00:00, 10.84it/s]


Epoch 2: Loss=1.8425, Genre F1=0.5148, Keyword F1=0.1346


...

Epoch 9: Loss=1.0032, Genre F1=0.6319, Keyword F1=0.2025


Evaluating: 100%|██████████| 68/68 [00:06<00:00, 10.92it/s]


Epoch 10: Loss=0.9775, Genre F1=0.6279, Keyword F1=0.2063


Evaluating: 100%|██████████| 68/68 [00:06<00:00, 10.98it/s]


Fold 5 - Genre F1 Micro: 0.6272, Keyword F1 Micro: 0.2118

Zapisano metryki wszystkich foldów do: results/cross_val_multitask_metrics.csv

=== Średnie wyniki 5-fold CV ===
loss: 1.8270
genre_f1_score: 0.6169
genre_f1_micro: 0.6242
genre_f1_macro: 0.5752
genre_precision_micro: 0.4982
genre_recall_micro: 0.8357
genre_hamming_loss: 0.1702
genre_hamming_accuracy: 0.8298
genre_jaccard_samples: 0.4796
keyword_f1_score: 0.2068
keyword_f1_micro: 0.2066
keyword_f1_macro: 0.2081
keyword_precision_micro: 0.1250
keyword_recall_micro: 0.5967
keyword_hamming_loss: 0.1641
keyword_hamming_accuracy: 0.8359
keyword_jaccard_samples: 0.1254

Średnie wyniki po 5-fold CV:
loss: 1.8270
genre_f1_score: 0.6169
genre_f1_micro: 0.6242
genre_f1_macro: 0.5752
genre_precision_micro: 0.4982
genre_recall_micro: 0.8357
genre_hamming_loss: 0.1702
genre_hamming_accuracy: 0.8298
genre_jaccard_samples: 0.4796
keyword_f1_score: 0.2068
keyword_f1_micro: 0.2066
keyword_f1_macro: 0.2081
keyword_precision_micro: 0.1250
keyword